## Cargando datos y paquetes

#### Librerías 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scrapy import Selector
import requests
from scrapy.crawler import CrawlerProcess
import scrapy
from scrapy.utils.project import get_project_settings

#### Cargando Dataset

In [ ]:
anime = pd.read_csv("../input/anime.csv")
rating = pd.read_csv("../input/rating.csv")

## Preprocesamiento data anime

### Explorando datos

En primera instancia se explora la existencia de casos duplicados, también la dimensión de la data y el tipo de dato de cada variable. Se observa que la variable "episodes" es de tipo object y no numérico dado que contiene una categoría "Unknown" que indica que no se conoce la cantidad de episodios del anime, es por esto, que se procede a cambiar "Unknown" por NaN y luego el tipo de dato a numérico.



In [ ]:
print(anime.shape)
print(anime.drop_duplicates().shape)
print(anime.info())
anime.tail(15)

In [ ]:
anime.replace("Unknown", np.nan, inplace=True)
anime["episodes"] = anime["episodes"].astype(float)

### Indentificando NaN 

Podemos detectar que las variables "genre", "type", "episodes" y "rating" poseen datos faltantes, donde "episodes" es la variable con más datos faltantes, seguida por "rating". Por otro lado, "genre" y "type" tienen mucho menos de missing. En total la data tiene 464 NaN.

In [ ]:
print(anime.isnull().sum())
print(anime[anime.isnull().any(axis=1)].shape)
anime[anime.isnull().any(axis=1)].head()



## Web Scraping para rellenar valores nulos

In [ ]:
nombres= anime[anime.isnull().any(axis=1)]
nombres = nombres["name"].values.tolist()

tipoAnime=pd.get_dummies(anime["type"]).columns
tipoAnime=tipoAnime.str.strip().unique().tolist()

genero=anime["genre"].str.get_dummies(sep=",").columns
genero=genero.str.strip().unique().tolist()

In [ ]:
buscarURL = 'https://myanimelist.net/search/all?q='
urlAnime = []
for i in nombres:
    urlAnime.append(buscarURL + i)


class AnimeFcSpider(scrapy.Spider):
    name = 'anime_fc'

    def start_requests(self):  # start_requests method
        for url2 in urlAnime:
            yield scrapy.Request(url=url2,
                                 callback=self.parse_front)

    def parse_front(self, response):  # First parsing method
        course_links = response.xpath('//div[@class="picSurround di-tc thumb"]/a/@href')
        yield response.follow(url=course_links[0],
                              callback=self.parse_pages)

    def parse_pages(self, response):  # Second parsing method
        crs_name = response.xpath('//h1[@class="h1"]/span/text()').extract_first()
        crs_episodes = response.xpath('//td[@class="spaceit"]/span[@id="curEps"]/text()').extract_first()
        crs_rating = response.xpath('//span[@itemprop="ratingValue"]/text()').extract_first()
        crs_id = response.xpath('//input[@name="aid"]/@value').extract_first()

        crs_genre = response.xpath('//div/a/@title').extract()
        crs_genre = np.intersect1d(crs_genre, genero)
        crs_genre = ','.join(map(str, crs_genre))

        crs_type = response.xpath('//div/a/text()').extract()
        crs_type = np.intersect1d(crs_type,tipoAnime)
        crs_type = ','.join(map(str, crs_type))
       

        list_name.append(crs_name)
        list_genre.append(crs_genre)
        list_type.append(crs_type)
        list_episodes.append(crs_episodes)
        list_rating.append(crs_rating)
        list_id.append(crs_id)



list_name = list()
list_genre = list()
list_type = list()
list_episodes = list()
list_rating = list()
list_id = list()

s = get_project_settings()
s['CONCURRENT_REQUESTS_PER_IP'] = 16
s['CONCURRENT_REQUESTS_PER_DOMAIN '] = 16
s['DOWNLOAD_DELAY'] = 2.5
s['CONCURRENT_REQUESTS'] = 32
s['CONCURRENT_REQUESTS'] = 32


process = CrawlerProcess(s)  # Run the Spider
process.crawl(AnimeFcSpider)
process.start()


In [ ]:
DataNa = pd.DataFrame({"anime_id":list_id, "name":list_name,"genre":list_genre,
                       "type":list_type, "episodes":list_episodes, "rating":list_rating})

DataNa.replace("", np.nan, inplace=True)
DataNa.replace('?', np.nan, inplace=True)

print(DataNa.shape)
print(DataNa.isnull().sum())
DataNa.head(10)

#### Preprocesando data obtenida con web scraping

Cambiamos el tipo de datos al mismo que la data original

In [ ]:
DataNa["anime_id"] = DataNa["anime_id"].astype(float)
DataNa["episodes"] = DataNa["episodes"].astype(float)
DataNa["rating"] = DataNa["rating"].astype(float)
DataNa.info()

Cruzamos la data obtenida con la origianal para luego reemplazar los nulos con la información nueva

In [ ]:
dataNueva= pd.merge(anime, DataNa,left_on="anime_id",right_on="anime_id", how="left")
dataNueva.info()
print(anime.isnull().sum())
print(anime[anime.isnull().any(axis=1)].shape)


In [ ]:
dataNueva.loc[dataNueva["genre_x"].isna(),"genre_x"] = dataNueva["genre_y"]
dataNueva.loc[dataNueva["type_x"].isna(),"type_x"] = dataNueva["type_y"]
dataNueva.loc[dataNueva["episodes_x"].isna(),"episodes_x"] = dataNueva["episodes_y"]
dataNueva.loc[dataNueva["rating_x"].isna(),"rating_x"] = dataNueva["rating_y"]


Eliminamos las varibles nuevas dado que ya utilizamos sus valores. 

In [ ]:
dataNueva.drop(["name_y", "genre_y", "type_y", "episodes_y", "rating_y"],axis=1,inplace=True)
dataNueva.columns = dataNueva.columns.str.replace('_x', '')

print(dataNueva.isnull().sum())
print(dataNueva[dataNueva.isnull().any(axis=1)].shape)

## Imputando datos nulos

#### Variable "episodes"

La primera variable a imputar es "episodes", dado que es la con mayor cantidad de NaN, para esto agruparemos por "type" y utilizaremos la mediana() de la cantidad de episodios de cada grupo.

In [ ]:
anime=dataNueva.copy()
print(anime.groupby("type")["episodes"].describe())

anime.loc[(anime["type"]=="OVA") & (anime["episodes"].isna()),"episodes"] = anime.loc[(anime["type"]=="OVA") ,"episodes"].median()
anime.loc[(anime["type"]=="Movie") & (anime["episodes"].isna()),"episodes"] = anime.loc[(anime["type"]=="Movie") ,"episodes"].median()
anime.loc[(anime["type"]=="Music") & (anime["episodes"].isna()),"episodes"] = anime.loc[(anime["type"]=="Music") ,"episodes"].median()
anime.loc[(anime["type"]=="ONA") & (anime["episodes"].isna()),"episodes"] = anime.loc[(anime["type"]=="ONA") ,"episodes"].median()
anime.loc[(anime["type"]=="Special") & (anime["episodes"].isna()),"episodes"] = anime.loc[(anime["type"]=="Special") ,"episodes"].median()
anime.loc[(anime["type"]=="TV") & (anime["episodes"].isna()),"episodes"] = anime.loc[(anime["type"]=="TV") ,"episodes"].median()
anime.loc[(anime["type"].isna()) & (anime["episodes"].isna()),"episodes"] = anime["episodes"].median()

print(anime[anime.isnull().any(axis=1)].shape)
print(anime.isnull().sum())

#### Variable "type"

Esta variable tiene 9 nulos, podríamos inferir el tipo por la cantidad de capítulos del animé, pero justamente estos 9 animé no tienen esa información por lo que reemplazaremos el dato nulo por "notype" para no eliminar la observación y así perder información valiosa. 


In [ ]:
anime["type"].replace(np.nan, "notype", inplace=True)
print(anime.isnull().sum())

#### Variable "genre"

Tenemos 45 animé con nulos en genero. pero dado que esta variable es muy importante en la elección del animé (por conocimiento propio) una imputación errónea sería grabe, por lo tanto haremos lo mismo que con "type" y crearemos una categoría para los nulos "nogenre"

In [ ]:
anime["genre"].replace(np.nan, "nogenre", inplace=True)
print(anime.isnull().sum())

#### Variable "rating"

Para esta variable haremos una imputación un poco más dirigida, se agrupará por "type" y "epidodes" y se calculará la mediana de rating con esa agrupación para imputar rating. En caso que los grupos "type" y "episodes" no tengan una mediana para "rating" se agrupará por "genre" y "epidodes" y si aún así no hay una mediana para "rating", entoces los datos nulos se reemplazarán por la mediana global. 

In [ ]:
def impute_median(series):
    return series.fillna(series.median())

anime.rating = anime.groupby(['type', 'episodes'])[["rating"]].transform(impute_median)
anime.rating = anime.groupby(['genre', 'episodes'])[["rating"]].transform(impute_median)
anime["rating"]=anime["rating"].fillna(anime["rating"].median())
print(anime.isnull().sum())


Se resetea el índice de la data para no tener problemas en el futuro para buscar filas especificas

In [ ]:
anime=anime.reset_index()

### Construyendo data con variables para análisis

#### Re-codificando variables

No era conveniente tener los géneros apilados como categoría separadas por comas en una única casilla, por lo que se separaron y pasaros a variables dicotómicas al igual que "type". Las variables restantes serán escaladas para no tener problemas con los algoritmos futuros, dado que utilizan distancias.

In [ ]:
anime_data = pd.concat([anime["genre"].str.get_dummies(sep=","),
                           anime["type"].str.get_dummies(sep=","),anime[["rating"]],
                            anime[["members"]],anime["episodes"]],axis=1)

anime_data.head()


In [ ]:
anime_data = MaxAbsScaler().fit_transform(anime_data)
anime_data

## Algoritmo no supervisado para encontrar elementos similares 
## Parte 1: Animes similares a un anime especifico

### K vecino más cercano (KNN)

El K-Vecino más cercano es la opción que me pareció más aceptada, dado que es un algoritmo jerárquico por lo cual no tenemos que elegir grupos a priori y es exactamente lo que estamos buscamos, explico: 

Lo que necesitamos es un algoritmo que tome un anime (cada fila representa un anime diferente) y de acuerdo a sus características (calificación, géneros, tipo y cantidad de episodios) pueda encontrar animes similares. KNN toma la distancia de una observación con cada observación de la data (importante tener los datos en la misma escala) y es exactamente lo que nos interesa rescatar, dado que es un claro indicador de similitud entre animes, además nos da la opción de elegir los k vecinos más próximos al animé buscado. 

(El objetivo no es encontrar clúster o grupos de animé, si no desde un punto establecido en el espacio rescatar los puntos más próximos)

Parámetros KNN

n_neighbors:
El número de vecinos solo nos agrega más elementos en la salida, es decir, n_neighbors=k sólo me indicará que "índices" tendrá un vector de k-1 elementos correspondiente los índices de los vecinos más cercano del anime consultado.  




In [ ]:
KNNanime = NearestNeighbors(n_neighbors=7, algorithm='ball_tree').fit(anime_data)
distances, indices = KNNanime.kneighbors(anime_data)

In [ ]:
def nombres_indices(name):  # Toma el nombre del anime y devuelve su indice correspondiente
    return anime[anime["name"]==name].index.tolist()[0] 


In [ ]:
def recomendados_por_anime(nombre):  # Muestra el grupo de animes más cercanos al consultado
     found_id = nombres_indices(nombre)
     for id in indices[found_id][1:]:
            print(anime.loc[id]["name"])
            
recomendados_por_anime("Naruto")
        
       

## Parte 2: Animes recomendados para cada usuario

En la parte 1 sólo conseguimos encontrar animes similares a otros animes, pero no estamos recomendando nada al usuario, es por esto, que utilizaremos la data riting.csv que contiene información del usuario para crear un recomendado de anime según preferencias del usuario utilizando las distancias de similitud obtenidas en la parte 1.

### Explorando data riting

Esta data contiene un id del usuario (user_id), el id del anime (anime_id) y la calificación que da el usuario al anime (rating).

No contiene NaN, pero la variable rating contiene el valor -1 que significa que el usuario no calificó el anime, esto puede ser considerado como un dato faltando.


In [ ]:
print(rating.shape)
print(rating.isnull().sum())
rating.head()

El siguiente paso es cruzar las datas anime y rating por la izquierda, dado que la data "rating" puede contener
animes que no se encuentran en la data "anime" y esto puede ser un problema en el futuro. 

In [ ]:
merge = pd.merge(anime, rating, on="anime_id", how="left")
merge.head()

### Construyendo recomendador

Necesitamos obtener todos los animes vistos por un usuario especifico, dado que según esto podemos capturar sus preferencias, luego de obtenidos los animes vistos por el usuario se procede a guardar en una lista con todos los animes similares a los que a visto el usuario (vecinos del algoritmo KNN ) excluyendo los que ha vistos (para no recomendar un anime que el usuario ya vio). Por último, se toma esta lista y se calcula la frecuencia de los animes que más se repiten en la lista y se ordenan de mayor a menor. 

La función ecomendados_usuario() devuelve los animes recomendados para el usuario.


In [ ]:
def similar_animes(id_anime):  # Trae todos los id_anime relacionados con un id_anime dado
    
    id_list=[]
    found_id = anime[anime["anime_id"]==id_anime].index.tolist()[0]  # Indice del id ingresado
    for id in indices[found_id][1:]:
            id_list.append(anime.loc[id]["anime_id"])
            
    return id_list  
        
            
def similar_animes_usuarios(id_user):  # Crea una lista con todos los animes relacionados con los animes visto por el usuario
    
    a = merge[merge["user_id"]==id_user].anime_id.values
    lista = []
    for i in range(len(a)):
        lista.append(similar_animes(a[i]))
    return lista
            
        
def similar_animes_usuarios_freq(id_user): # Crea una lista con los 6 anime más recomendados del usuario
    a=similar_animes_usuarios(id_user)
    r= np.array([])
    for i in range(5):
        f1 = pd.Series( (v[i] for v in a))
        r = np.append(r,f1)
        
    gh = merge[merge["user_id"]==id_user].anime_id.values
    rdiff=np.setdiff1d(r, gh)
    kk = pd.DataFrame({'Column1':rdiff})
    pda = pd.crosstab(index=kk["Column1"].astype(int), columns= "count")
    pda2 = pda.sort_values("count", ascending=False).head(6).index.tolist() 
    
    return pda2
        
    
def recomendados_usuario(id_user):  # Pasa de anime_id a los nombres de los animé
    
    a=similar_animes_usuarios_freq(id_user)
    for id in a:
        print(anime[anime["anime_id"]==id]["name"].values)
        


### Utilizando funciones de recomendación

#### Animes recomendados por usuario

In [ ]:
recomendados_usuario(3454)

In [ ]:
recomendados_usuario(8765)

#### Animes recomendados por anime

In [ ]:
recomendados_por_anime("Dragon Ball Z")

In [ ]:
recomendados_por_anime("Pokemon")